# Projet Question Answering - Annexes

Ne pas hésiter à retélécharger ce fichier régulièrement pour voir si le
contenu a évolué.

## Installation de pyserini

L’installation est assez délicate, les instruction suivantes sont
inspirées de
<https://github.com/castorini/pyserini/blob/master/docs/installation.md>:

``` shell
conda create -n pyserini python=3.10 -y
conda activate pyserini
conda install -c conda-forge openjdk=11 maven -y
conda install -c conda-forge lightgbm nmslib -y
conda install -c pytorch faiss-cpu mkl=2021 blas=1.0=mkl pytorch -y
pip install pyserini
```

Une version sans conda devrait marcher aussi, à condition d’avoir Python
3.10 et pas une version plus récente:

``` shell
python -m venv serini
source serini/bin/activate
sudo apt update
sudo apt install default-jdk build-essential
pip install faiss-cpu
pip install torch --index-url https://download.pytorch.org/whl/cu118
pip install pyserini
```

## Installation des modules Huggingface

``` shell
pip install datasets
pip install transformers
```

Ces deux bibliothèques gèrent le téléchargement des datasets et des
modèles pré-entraînés. Le téléchargement se fait une fois pour toute
lors de la première exécution.

## Installation de Langchain

``` shell
pip install langchain
```

## Chargement du dataset `nq_open`

In [1]:
from datasets import load_dataset

dataset = load_dataset("nq_open", split="validation")
for batch in dataset.iter(batch_size=1):
    print(batch["question"], batch["answer"])

ModuleNotFoundError: No module named 'datasets'

## Modèle probabiliste avec Pyserini

In [ ]:
import json
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('wikipedia-dpr-100w')
hits = searcher.search('How old is Harrisson Ford ?')

for i in range(0, 10):
    docid = hits[i].docid
    score = hits[i].score
    content = json.loads(searcher.doc(docid).raw())["contents"]

    print(score, docid, content)

Cet index a été calculé à partir d’une extraction complète de wikipedia.
Chaque article a été découpé en passages d’une longueur de 100 mots. Un
document de l’index est l’un de ces passages.

## Modèle dense avec Pyserini

Ne pas l’utiliser tel quel, c’est trop coûteux pour une machine
personnelle, ce fragment est juste donné à titre de référence. On voit
ici l’encodeur de question
`facebook/dpr-question_encoder-single-nq-base` et l’index précalculé
avec l’encodeur de documents

In [ ]:
# from pyserini.search.faiss import FaissSearcher, DprQueryEncoder
# 
# encoder = DprQueryEncoder('facebook/dpr-question_encoder-single-nq-base')
# searcher = FaissSearcher.from_prebuilt_index(
#     'wikipedia-dpr-100w.dpr-single-nq',
#     encoder
# )
# hits = searcher.search('what is a lobster roll')

Il s’agit des mêmes documents que précédemment.

## Modèle dense avec Fakesearch

Il faut télécharger le fichier `fakesearch.py` et le fichier
`faiss-validation.pickle`. Les résultats de recherche ont été
précalculés et stockés dans un fichier.

In [ ]:
import fakesearch

dense_index = fakesearch.load("faiss-lucene.pickle")
dense_index["what is a lobster roll"]

Attention, on ne peut utiliser que les questions du sous-ensemble de
validation. On obtiendra une erreur dans le cas contraire:

In [ ]:
dense_index["How old is Harrison Ford"]

## Modèle probabiliste avec Fakesearch

La recherche pré-calculée n’est à utiliser que si vous n’arrivez
vraiment pas à installer `pyserini`.

Il faut télécharger le fichier `fakesearch.py` et le fichier
`lucence-validation.pickle`. Les résultats de recherche ont été
précalculés et stockés dans un fichier.

In [ ]:
import fakesearch

sparse_index = fakesearch.load("lucene-validation.pickle)
sparse_index["what is a lobster roll"]

Attention, on ne peut utiliser que les questions du sous-ensemble de
validation. On obtiendra une erreur dans le cas contraire:

In [ ]:
sparse_index["How old is Harrison Ford"]

## ChatGPT avec le module OpenAI

Cet interface est relativement simple, mais limitée à ChatGPT et ne
fournit aucun outil pour faciliter l’utilisation des réponses.

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=LA_CLÉ_ICI)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Who is Joe Biden ?",
        }
    ],
    model="gpt-3.5-turbo",
)

## ChatGPT avec Langchain

Langchain est un peu plus complexe à utiliser, mais permet d’utiliser
beaucoup plus d’outils pratiques et de s’abstraire du modèle de langue
utilisé.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=LA_CLÉ_ICI)
llm.invoke("what is a lobster roll")

## Duckduckgo avec Langchain

Voir <https://python.langchain.com/docs/integrations/tools/ddg>

In [ ]:
%pip install --upgrade --quiet  duckduckgo-search

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.run("what is a lobster roll")

## Llama2

Pour Llama2, on a besoin de:

-   Ollama <https://ollama.ai/> pour télécharger et faire tourner le
    modèle
-   Langchain pour l’utiliser

Voir <https://python.langchain.com/docs/integrations/chat/ollama>

## GPT2

Voir <https://huggingface.co/gpt2>

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="gpt2")
pipe("what is a lobster roll")